<a href="https://colab.research.google.com/github/alanntl/SELGO-LITE/blob/alanntl/SLEGO_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNSW FAIC SLEGO LITE
by Alan Ng

https://unsw-my.sharepoint.com/:w:/g/personal/z5380623_ad_unsw_edu_au/ESQwXO2CQulHoEfQiOHECq8B_hT7_Eqa-mH_Whn_goXGVw?e=3DUUMA

# Import libraries
Import all the related libraries

In [1]:
!pip install boto3

from IPython.display import clear_output
clear_output()
import panel as pn
from google.colab import files
import importlib
import os
import sys
import inspect
import ast  # For safely evaluating the input string
from google.colab import userdata
import boto3

pn.extension(sizing_mode = 'stretch_both')
pn.extension(comms='colab')

# Connect with cloud storage

In [2]:
# if you use AWS S3, Please inpout your secret keys

# Set your AWS credentials as environment variables
os.environ['AWS_ACCESS_KEY_ID'] =  userdata.get('aws_access_key_id')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('aws_secret_access_key')
# Optionally, set a default region (e.g., 'ap-southeast-2' for Sydney)
os.environ['AWS_DEFAULT_REGION'] = userdata.get('AWS_DEFAULT_REGION')

bucket_name =  userdata.get('bucket_name')# Replace with your bucket name
s3 = boto3.client('s3')


# Get cloud data

In [3]:
def merge_s3_folder_to_py(s3, bucket_name, prefix):
    # List all objects within a bucket path
    objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Merge the content of Python files
    merged_content = ""

    for obj in objects['Contents']:
        filename = obj['Key']
        if filename.endswith("/"):  # Don't try to download directories
            continue

        print(f'Merging {filename}...')

        try:
            # Download the content of the file
            response = s3.get_object(Bucket=bucket_name, Key=filename)
            file_content = response['Body'].read().decode('utf-8')

            # Append the content to the merged content
            merged_content += file_content
            merged_content += "\n\n"  # Add two newlines between scripts
        except Exception as e:
            print(f"Error merging {filename}: {str(e)}")

    return merged_content

bucket_name = bucket_name # Replace with your bucket name
prefix ='function'
merged_content = merge_s3_folder_to_py(s3, bucket_name, prefix)

# Save the merged content to a local .py file
with open('func.py', 'w', encoding='utf-8') as outfile:
    outfile.write(merged_content)


Merging function/func_EventStudy.py...
Merging function/func_HF.py...
Merging function/func_adage3.py...
Merging function/func_chatbot_v1.py...
Merging function/func_env.py...
Merging function/func_pipeline101.py...
Merging function/func_viz.py...
Merging function/func_yfinance.py...
Merging function/rand.py...
Merging function/vbt_backtest.py...


# Upload your local function file
You will need to upload your microservices file "func.py" to enable the services. \
you can download the "func.py" file from: \
https://drive.google.com/file/d/1JKtJDQY39Oxq6_AOGrU6Fw2UAFrJWK0G/view?usp=drive_link


In [4]:
# # Step 1: Create the File Upload Widget
# uploaded = files.upload()

# # Step 2: Save the Uploaded File
# uploaded_file_name = None
# for filename in uploaded.keys():
#     # Check if the file already exists
#     if os.path.exists(filename):
#         print(f"File {filename} already exists. Deleting it.")
#         os.remove(filename)

#     with open(filename, 'wb') as f:
#         f.write(uploaded[filename])
#     uploaded_file_name = filename.split('.')[0]  # Capture the module name without extension

# # Step 3: Import the Functions Dynamically
# if uploaded_file_name:
#     # If the module was imported before, reload it to get the new changes
#     if uploaded_file_name in globals():
#         imported_module = importlib.reload(globals()[uploaded_file_name])
#     else:
#         imported_module = importlib.import_module(uploaded_file_name)


# # Now you can use the functions defined in the uploaded .py file
# # For example, if there's a function named 'hello' in the uploaded file:
# # result = uploaded_module.hello()


# Import your uploaded services (lbirary)

In [7]:
!pip install quantstats vectorbt backtrader pyfolio dtale yahoo_fin
import func

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=5980c754cf55d16ea2a3278dedcd86f206c2c42f2b9ecc9f4c97091a588a69ac
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


/usr/local/lib/python3.10/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


# A simple interface to create data pipleline

The exmaple uses text input and Python Dictionaries to form a data pipeline. First download data from yahoo finance API, then use the downloaded data to calculate the moving average values of a specific column. You can easily change the input parameters for different results.

In [16]:
import inspect
import param
def list_functions(module):
    return [name for name, obj in inspect.getmembers(module) if inspect.isfunction(obj) and not name.startswith('__')]

functions = list_functions(func)

def select_func_change(event):
  text_input.value = str(func_multi_choice.value)

func_multi_choice = pn.widgets.MultiChoice(name='MultiSelect', options=functions, height=1000)
func_multi_choice.param.watch(select_func_change, 'value')




Watcher(inst=MultiChoice(height=1000, name='MultiSelect', options=['accuracy_score', ...], sizing_mode='stretch_width'), cls=<class 'panel.widgets.select.MultiChoice'>, fn=<function select_func_change at 0x7dec51dd4af0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [17]:
func_multi_choice

MultiChoice(height=1000, name='MultiSelect', options=['accuracy_score', ...], sizing_mode='stretch_width')

In [18]:
import importlib
importlib.reload(func)

# Initialize the text area input and button
functions = [(name, obj) for name, obj in vars(func).items() if callable(obj) and not name.startswith("__")]


compute_button = pn.widgets.Button(name='Compute', button_type='primary',height=50)
output_text= pn.widgets.TextAreaInput(value='',placeholder= 'Results will be shown here')
text_input = pn.widgets.TextAreaInput(value='''{
    "get_stock_data_101": {
        "ticker": "AAPL",
        "start_date": "2020-01-01",
        "end_date": "2021-01-01",
        "output_s3_file_key": "data/AAPL_stock_data.csv"
    },
    "preprocess_stock_data_101": {
        "input_s3_file_key": "data/AAPL_stock_data.csv",
        "output_s3_file_key": "data/AAPL_stock_data_processed.csv"
    },
    "compute_simple_moving_average_101": {
        "input_s3_file_key": "data/AAPL_stock_data_processed.csv",
        "window_size": 20,
        "output_s3_file_key": "data/AAPL_stock_data_SMA.csv"
    },
    "plot_stock_data_101": {
        "input_s3_file_key": "data/AAPL_stock_data_SMA.csv",
        "ticker": "AAPL",
        "output_html_file_key": "plots/AAPL_stock_plot.html"
    }
}''', height=500)

# Define the callback function
def compute_callback(event):
    output_text.value = ''
    # Get the text from the text_input
    text = text_input.value
    # You can now do something with the text, like processing it or displaying results
    # For this example, let's just print it (or display it somewhere in your panel app)
    pipeline_dict = ast.literal_eval(text)
    for function_name, parameters in pipeline_dict.items():
        module = 'func.'
        function = eval(module+function_name)
        result = function(**parameters)

        output_text.value += "\n===================="+function_name+"====================\n\n"
        output_text.value += str(result)


# Link the button click event to the callback function
compute_button.on_click(compute_callback)

# Create a panel layout and show it
app =pn.Row( pn.Column(text_input,compute_button) , output_text)

app

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_both')
        [0] TextAreaInput(height=500, sizing_mode='stretch_width', value='{\n    "get_stock_data_10...)
        [1] Button(button_type='primary', height=50, name='Compute', sizing_mode='stretch_width')
    [1] TextAreaInput(placeholder='Results will b..., sizing_mode='stretch_both')